## Case Study: Stock Price Prediction 


<h4> The Case study is to predict Next day stock price [Adj Close Price] using  regression analysis</h4>
<p>
    We will be using Nifty 50 data for this case study<br>
    The filename '^NSEI.csv' contains the data<br>
    You need to generate  technical indicators as given below as  Features for converting Time Series to Supervised Learning Problem <br>
   <a href='https://machinelearningmastery.com/convert-time-series-supervised-learning-problem-python/'> Convert time Series to Supervised Learning</a><p>
    <ul>
        <li> Moving Averages (Example Given : 3Period Moving Avearage (3MA)) </li>
        <li> Previous Day Returns </li>
        <li> Relative Stock returns </li>
    </ul>
<br>
Then you need to Train a Supervised learning algorithm and perform prediction.<br>
<a href='https://blog.quantinsti.com/random-forest-algorithm-in-python/'>Example for your Refrence </a><br>
<b> Make sure  to shift the feature by 1 period so that you will have data from t-1 period to predict price at t (look ahead bias) </b>

<strong> Please consider Testing Period after  01-01-2019 </strong>
</p>
<p>
    <b> Metrics for Evaluation </b>
    <ul>
        <li><strong> RMSE </strong></li>
     </ul>
</p>
<p> Some additional Documention for your help
    <ul>
        <li><a href='https://pandas.pydata.org/docs/'> Pandas Documentation </a> </li>
        <li><a href='https://scikit-learn.org/stable/'> Scikit Learn </a></li>
        </ul>
        </p>

In [1]:
import pandas as pd
import numpy as np

In [2]:
# Reading the Stock 
datos=pd.read_csv("^NSEI.csv")
datos['Date'] = pd.to_datetime(datos['Date'])
datos = datos.set_index('Date')

In [3]:
# Getting the head of the data
datos.head()

Open         High          Low        Close    Adj Close  \
Date                                                                          
2007-09-17  4518.450195  4549.049805  4482.850098  4494.649902  4494.649902   
2007-09-18  4494.100098  4551.799805  4481.549805  4546.200195  4546.200195   
2007-09-19  4550.250000  4739.000000  4550.250000  4732.350098  4732.350098   
2007-09-20  4734.850098  4760.850098  4721.149902  4747.549805  4747.549805   
2007-09-21  4752.950195  4855.700195  4733.700195  4837.549805  4837.549805   

            Volume  
Date                
2007-09-17     0.0  
2007-09-18     0.0  
2007-09-19     0.0  
2007-09-20     0.0  
2007-09-21     0.0

In [4]:
#### Generating a Sample Feature for your Reference 3period moving average
datos['3MA']=datos['Adj Close'].rolling(3).mean()
### shifting Feature 3MA  so that features at t-1 will be used to predict close at t
datos['3MA']=datos['3MA'].shift(1)

In [5]:
datos.head()

Open         High          Low        Close    Adj Close  \
Date                                                                          
2007-09-17  4518.450195  4549.049805  4482.850098  4494.649902  4494.649902   
2007-09-18  4494.100098  4551.799805  4481.549805  4546.200195  4546.200195   
2007-09-19  4550.250000  4739.000000  4550.250000  4732.350098  4732.350098   
2007-09-20  4734.850098  4760.850098  4721.149902  4747.549805  4747.549805   
2007-09-21  4752.950195  4855.700195  4733.700195  4837.549805  4837.549805   

            Volume          3MA  
Date                             
2007-09-17     0.0          NaN  
2007-09-18     0.0          NaN  
2007-09-19     0.0          NaN  
2007-09-20     0.0  4591.066732  
2007-09-21     0.0  4675.366699

In [6]:
# Generate More technical Features
datos['Adj Close t-1'] = datos['Adj Close'].shift(1)
datos['Open-Close'] = (datos.Open - datos.Close)/datos.Open
datos['Open-Close']=datos['Open-Close'].shift(1)
datos['High-Low'] = (datos.High - datos.Low)/datos.Low
datos['High-Low']=datos['High-Low'].shift(1)
datos['percent_change'] = datos['Adj Close'].pct_change()
datos['percent_change']=datos['percent_change'].shift(1)
datos['std_3'] = datos['percent_change'].rolling(3).std()
datos['std_3']=datos['std_3'].shift(1)
datos['ret_3'] = datos['percent_change'].rolling(3).mean()
datos['ret_3']=datos['ret_3'].shift(1)
datos.dropna(inplace=True)
datos.head()

Open         High          Low        Close    Adj Close  \
Date                                                                          
2007-09-24  4837.149902  4941.149902  4837.149902  4932.200195  4932.200195   
2007-09-25  4939.100098  4953.899902  4878.149902  4938.850098  4938.850098   
2007-09-26  4937.600098  4980.850098  4930.350098  4940.500000  4940.500000   
2007-09-27  4942.700195  5016.399902  4942.700195  5000.549805  5000.549805   
2007-09-28  4996.450195  5055.799805  4996.450195  5021.350098  5021.350098   

            Volume          3MA  Adj Close t-1  Open-Close  High-Low  \
Date                                                                   
2007-09-24     0.0  4772.483236    4837.549805   -0.017799  0.025773   
2007-09-25     0.0  4839.099935    4932.200195   -0.019650  0.021500   
2007-09-26     0.0  4902.866699    4938.850098    0.000051  0.015528   
2007-09-27     0.0  4937.183431    4940.500000   -0.000587  0.010243   
2007-09-28     0.0  4959.966634    5000.549805   -0.011704  0.014911   

            percent_change     std_3     ret_3  
Date                                            
2007-09-24        0.018957  0.019837  0.018542  
2007-09-25        0.019566  0.018953  0.021038  
2007-09-26        0.001348  0.009271  0.013912  
2007-09-27        0.000334  0.010347  0.013290  
2007-09-28        0.012155  0.010823  0.007083

In [7]:
# Train Test Split  data (Make sure to consider the data is a time series data)
datos['Year'] = datos.index.year
datos_train = datos[datos['Year'] < 2019]
datos_test = datos[datos['Year'] > 2018]
datos_train.head()

Open         High          Low        Close    Adj Close  \
Date                                                                          
2007-09-24  4837.149902  4941.149902  4837.149902  4932.200195  4932.200195   
2007-09-25  4939.100098  4953.899902  4878.149902  4938.850098  4938.850098   
2007-09-26  4937.600098  4980.850098  4930.350098  4940.500000  4940.500000   
2007-09-27  4942.700195  5016.399902  4942.700195  5000.549805  5000.549805   
2007-09-28  4996.450195  5055.799805  4996.450195  5021.350098  5021.350098   

            Volume          3MA  Adj Close t-1  Open-Close  High-Low  \
Date                                                                   
2007-09-24     0.0  4772.483236    4837.549805   -0.017799  0.025773   
2007-09-25     0.0  4839.099935    4932.200195   -0.019650  0.021500   
2007-09-26     0.0  4902.866699    4938.850098    0.000051  0.015528   
2007-09-27     0.0  4937.183431    4940.500000   -0.000587  0.010243   
2007-09-28     0.0  4959.966634    5000.549805   -0.011704  0.014911   

            percent_change     std_3     ret_3  Year  
Date                                                  
2007-09-24        0.018957  0.019837  0.018542  2007  
2007-09-25        0.019566  0.018953  0.021038  2007  
2007-09-26        0.001348  0.009271  0.013912  2007  
2007-09-27        0.000334  0.010347  0.013290  2007  
2007-09-28        0.012155  0.010823  0.007083  2007

In [8]:
X_train = datos_train[['3MA', 'Adj Close t-1', 'Open-Close', 'percent_change', 'High-Low', 'std_3', 'ret_3']]
Y_train = datos_train[['Adj Close']]
X_test = datos_test[['3MA', 'Adj Close t-1', 'Open-Close', 'percent_change', 'High-Low', 'std_3', 'ret_3']]
Y_test = datos_test[['Adj Close']]

In [9]:
# Train different Algorithms
import statsmodels.api as sm
X_train=sm.add_constant(X_train)
results = sm.OLS(Y_train, X_train).fit()
resumen= results.summary()
X_test=sm.add_constant(X_test)
Y_predict = results.predict(X_test)

In [10]:
# Test performance of the model on the Testing Data using RMSE
from sklearn.metrics import mean_squared_error
from math import sqrt

rms = sqrt(mean_squared_error(Y_test, Y_predict))
print(f'The value of RMSE is {rms}')

The value of RMSE is 99.89615474647375
